In [23]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

In [24]:
df = pd.read_parquet("hf://datasets/cornell-movie-review-data/rotten_tomatoes/train.parquet")
texts = df["text"]
labels = df["label"]
x_train, x_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [25]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(x_test.tolist(), truncation=True, padding=True, max_length=128)

In [26]:
class MovieReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MovieReviewDataset(train_encodings, y_train)
test_dataset = MovieReviewDataset(test_encodings, y_test)


In [27]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

In [28]:
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # 训练快一点
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    save_strategy="no",  # 不自动保存中间模型
    logging_dir='./logs',
    logging_steps=50,
    report_to="none"
)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [31]:
trainer.train()

Step,Training Loss
50,0.689400
100,0.560200
150,0.514900
200,0.472100
250,0.447500
300,0.431600
350,0.474100
400,0.443500
450,0.426300
500,0.435600


TrainOutput(global_step=853, training_loss=0.44199030983489956, metrics={'train_runtime': 3196.7451, 'train_samples_per_second': 2.135, 'train_steps_per_second': 0.267, 'total_flos': 137712279721536.0, 'train_loss': 0.44199030983489956, 'epoch': 1.0})

In [32]:
eval_results = trainer.evaluate()
print("📊 Final Evaluation:", eval_results)

📊 Final Evaluation: {'eval_loss': 0.3362458646297455, 'eval_accuracy': 0.8651817116060961, 'eval_f1': 0.8651490952347507, 'eval_runtime': 181.9135, 'eval_samples_per_second': 9.378, 'eval_steps_per_second': 0.297, 'epoch': 1.0}


In [33]:
# 用模型预测一句话
inputs = tokenizer("This movie is amazing!", return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
pred = torch.argmax(outputs.logits)
print("预测类别：", pred.item())


预测类别： 1


In [34]:
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_tokenizer")
#保存或重复使用

('./sentiment_tokenizer/tokenizer_config.json',
 './sentiment_tokenizer/special_tokens_map.json',
 './sentiment_tokenizer/vocab.txt',
 './sentiment_tokenizer/added_tokens.json',
 './sentiment_tokenizer/tokenizer.json')